In [1]:
#!/usr/bin/env python
# -*- coding: latin-1 -*-
import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import logging
import difflib
import Levenshtein as lev
from tqdm.notebook import tqdm
import multiprocessing
from gensim.models import Doc2Vec
import gensim.models.doc2vec
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy import spatial
import re
import movies_df_helper_functions as helper_fn
import random
import imp
from sklearn.manifold import TSNE
from sklearn.cluster import AffinityPropagation
from tabulate import tabulate
import decimal
from collections import Counter
import evaluation_helper_functions as eval_fn

import itertools
imp.reload(helper_fn)
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-2-63d8866e010a>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [3]:

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
tqdm.pandas()

d:\programs\python39\lib\site-packages\tqdm\std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
# load movies.csv
movies_file_df = pd.read_csv('imdb_final_dataset.csv')

In [5]:
movies_file_df.describe

<bound method NDFrame.describe of          movie_id                       movie_name                genres  \
0       tt0000615  Robbery Under Arms               Drama                  
1       tt0000630  Hamlet                           Drama                  
2       tt0000676  Don Álvaro o la fuerza del sino  Drama                  
3       tt0000886  Hamlet, Prince of Denmark        Drama                  
4       tt0000941  Locura de amor                   Drama                  
...           ...             ...                     ...                  
187645  tt9916170  The Rehearsal                    Drama                  
187646  tt9916270  Il talento del calabrone         Thriller               
187647  tt9916362  Coven                            Drama,History,Horror   
187648  tt9916538  Kuambil Lagi Hatiku              Drama                  
187649  tt9916706  Dankyavar Danka                  Comedy                 

                                                     

In [6]:
movies_file_df.shape

(187650, 6)

In [7]:
movies_file_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187650 entries, 0 to 187649
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   movie_id       187650 non-null  object
 1   movie_name     187650 non-null  object
 2   genres         187650 non-null  object
 3   actor_name     187650 non-null  object
 4   director_name  187650 non-null  object
 5   writer_name    187650 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [8]:
movies_file_df.describe(include=['object', 'bool'])

,movie_id,movie_name,genres,actor_name,director_name,writer_name
count,187650,187650,187650,187650,187650,187650
unique,187650,170042,1134,186060,67987,137787
top,tt0357009,Hamlet,Drama,Peter Coyote,Sam Newfield,William Shakespeare
freq,1,25,43943,13,202,243


In [9]:
movies_file_df[movies_file_df['movie_id'].duplicated()]

,movie_id,movie_name,genres,actor_name,director_name,writer_name


***********************Prepare the movies information to be used for training the doc2vec model ***********************
\rnsteps:
Step 1-split actor_name, director_name,writer_name,genere_id,country_id and convert it to list of words
#Step 2-concatenate first and last names with an _
#Step 3.convert all data to lower case
#Step 4-concatenate movie_id and title to overcome the issue of movies having the same title
#Step 5-create a sentence to represent each movie by combining all attributes for every movie in one string with space separating each value


In [10]:
#copy data into new dataframe
movies_df = movies_file_df.copy()

In [11]:
movies_df['genres'] = movies_df['genres'].apply(
            lambda row: row.lower().strip().replace(' ','_').split(','))
movies_df['actor_name'] = movies_df['actor_name'].apply(
            lambda row: row.lower().strip().replace(' ','_').split('|'))
movies_df['director_name'] = movies_df['director_name'].apply(
            lambda row: row.lower().strip().replace(' ','_').split('|'))
movies_df['writer_name'] = movies_df['writer_name'].apply(
            lambda row: row.lower().strip().replace(' ','_').split('|'))
movies_df['movie_id'] = movies_df.apply(
            lambda row: row['movie_name'].lower().strip().replace(' ','_')  + '_' + row['movie_id'], axis=1)
movies_df = movies_df.rename(columns={'movie_name':'title', 'genres':'genre'})
movies_df['title']=movies_df['title'].str.lower()

In [12]:
#movies_df.describe(include=['object', 'bool'])

In [13]:
#x = movies_df['movie_name'].lower().replace(' ','_') + '_' + movies_df['movie_id']



In [14]:
movies_df.tail(10)

,movie_id,title,genre,actor_name,director_name,writer_name
187640,the_new_estate_tt9914808,the new estate,"[adventure, horror]","[ryan_johnson, brad_mcmurray]",[peter_andrikidis],"[lisa_campos, alexander_falzon]"
187641,the_war_of_godzilla_tt9914828,the war of godzilla,"[action, comedy, family]","[michael, the_man, mario, liam]",[kyle],"[willis, jake, hiro, nathan, mike]"
187642,la_vida_sense_la_sara_amat_tt9914942,la vida sense la sara amat,[drama],"[biel_rossell_pelfort, isaac_alcayde, maria_morera_colomer, lluís_altés]",[laura_jou],"[pep_puig, coral_cruz]"
187643,the_heart_to_fight_tt9915010,the heart to fight,"[action, thriller]","[shannon_taylor, andrei_gromyko, andrew_biernat, brennon_peters]","[jocelyn_sevier, lolly_boroff, michael_jones]",[simon_k._parker]
187644,the_last_white_witch_tt9915872,the last white witch,"[comedy, drama, fantasy]",[fumika_shimizu],[hideki_kiyota],[ryuho_okawa]
187645,the_rehearsal_tt9916170,the rehearsal,[drama],"[kelner_macêdo, pablo_lafuente, julia_ianina, germano_melo]",[tamar_guimaraes],"[melissa_de_raaf, lillah_halla]"
187646,il_talento_del_calabrone_tt9916270,il talento del calabrone,[thriller],"[anna_foglietta, lorenzo_richelmy, sergio_castellitto, marina_occhionero]",[giacomo_cimini],[lorenzo_collalti]
187647,coven_tt9916362,coven,"[drama, history, horror]","[alex_brendemühl, daniel_fanego, amaia_aberasturi, garazi_urkola]",[pablo_agüero],[katell_guillou]
187648,kuambil_lagi_hatiku_tt9916538,kuambil lagi hatiku,[drama],"[cut_mini_theo, sahil_shah, lala_karmela, ria_irawan]",[azhar_kinoi_lubis],"[rino_sarjono, arief_ash_siddiq, salman_aristo]"
187649,dankyavar_danka_tt9916706,dankyavar danka,[comedy],"[ashwini_ekbote, suvarna_kale, makarand_anaspure, prakash_dhotre, anvay_bendre]",[kanchan_nayak],[sudhir_nikam]


In [15]:
display(tabulate(movies_df.head(), tablefmt='html'))

0,robbery_under_arms_tt0000615,robbery under arms,['drama'],"['jim_gerald', 'george_merriman', 'lance_vane', 'arthur_guest', 'rhoda_dendron', 'william_duff']",['charles_macmahon'],['rolf_boldrewood']
1,hamlet_tt0000630,hamlet,['drama'],['fernanda_negri_pouget'],['mario_caserini'],['william_shakespeare']
2,don_álvaro_o_la_fuerza_del_sino_tt0000676,don álvaro o la fuerza del sino,['drama'],"['joaquín_carrasco', 'jaime_borrás']",['narciso_cuyàs'],['duque_de_rivas']
3,"hamlet,_prince_of_denmark_tt0000886","hamlet, prince of denmark",['drama'],['jean_mounet-sully'],['gérard_bourgeois'],['william_shakespeare']
4,locura_de_amor_tt0000941,locura de amor,['drama'],"['joaquín_carrasco', 'josé_argelagués', 'elvira_fremont', 'josé_durany']","['alberto_marro', 'ricardo_de_baños']",['manuel_tamayo_y_baus']


In [16]:
#movies_df['movie_sentence'] = movies_df.progress_apply(lambda row:  helper_fn.build_sentence_from_col(row.actor_name, row.genre, row.director_name, row.writer_name,''), axis=1)

movies_df['movie_sentence']= movies_df.actor_name + movies_df.genre + movies_df.director_name + movies_df.writer_name



In [17]:
print(movies_df[0:5])

                                    movie_id                            title  \
0  robbery_under_arms_tt0000615               robbery under arms                
1  hamlet_tt0000630                           hamlet                            
2  don_álvaro_o_la_fuerza_del_sino_tt0000676  don álvaro o la fuerza del sino   
3  hamlet,_prince_of_denmark_tt0000886        hamlet, prince of denmark         
4  locura_de_amor_tt0000941                   locura de amor                    

     genre  \
0  [drama]   
1  [drama]   
2  [drama]   
3  [drama]   
4  [drama]   

                                                                             actor_name  \
0  [jim_gerald, george_merriman, lance_vane, arthur_guest, rhoda_dendron, william_duff]   
1  [fernanda_negri_pouget]                                                                
2  [joaquín_carrasco, jaime_borrás]                                                       
3  [jean_mounet-sully]                                          

In [18]:
#check if any movie has empty sentence
movies_df[movies_df['movie_sentence'].str.len() == 0].count()

movie_id          0
title             0
genre             0
actor_name        0
director_name     0
writer_name       0
movie_sentence    0
dtype: int64

In [19]:
movies_df['movie_sentence']

0         [jim_gerald, george_merriman, lance_vane, arthur_guest, rhoda_dendron, william_duff, drama, charles_macmahon, rolf_boldrewood]
1         [fernanda_negri_pouget, drama, mario_caserini, william_shakespeare]                                                           
2         [joaquín_carrasco, jaime_borrás, drama, narciso_cuyàs, duque_de_rivas]                                                        
3         [jean_mounet-sully, drama, gérard_bourgeois, william_shakespeare]                                                             
4         [joaquín_carrasco, josé_argelagués, elvira_fremont, josé_durany, drama, alberto_marro, ricardo_de_baños, manuel_tamayo_y_baus]
                                                                       ...                                                              
187645    [kelner_macêdo, pablo_lafuente, julia_ianina, germano_melo, drama, tamar_guimaraes, melissa_de_raaf, lillah_halla]            
187646    [anna_foglietta, lorenzo_richel

In [20]:
movies_df[18:]

,movie_id,title,genre,actor_name,director_name,writer_name,movie_sentence
18,anfisa_tt0002022,anfisa,[drama],"[vladimir_shaternikov, vladimir_maksimov]",[yakov_protazanov],[leonid_andreyev],"[vladimir_shaternikov, vladimir_maksimov, drama, yakov_protazanov, leonid_andreyev]"
19,cleopatra_tt0002101,cleopatra,"[drama, history]","[james_r._waite, miss_fielding, helene_costello, charles_sindelar, mr._howard, miss_robson, pearl_sindelar, helen_gardner]",[charles_l._gaskill],[victorien_sardou],"[james_r._waite, miss_fielding, helene_costello, charles_sindelar, mr._howard, miss_robson, pearl_sindelar, helen_gardner, drama, history, charles_l._gaskill, victorien_sardou]"
20,dante's_inferno_tt0002130,dante's inferno,"[adventure, drama, fantasy]","[arturo_pirovano, salvatore_papa, pier_delle_vigne, giuseppe_de_liguoro]","[francesco_bertolini, adolfo_padovan]",[dante_alighieri],"[arturo_pirovano, salvatore_papa, pier_delle_vigne, giuseppe_de_liguoro, adventure, drama, fantasy, francesco_bertolini, adolfo_padovan, dante_alighieri]"
21,the_great_circus_catastrophe_tt0002153,the great circus catastrophe,[drama],"[lauritz_olsen, agnes_lorentzen, frederik_christensen, valdemar_psilander, alma_hinding, aage_hertel, frederik_jacobsen, axel_mattsson]",[eduard_schnedler-sørensen],[alfred_kjerulf],"[lauritz_olsen, agnes_lorentzen, frederik_christensen, valdemar_psilander, alma_hinding, aage_hertel, frederik_jacobsen, axel_mattsson, drama, eduard_schnedler-sørensen, alfred_kjerulf]"
22,eva_tt0002163,eva,[drama],"[henny_porten, frida_richard, hans_marr, harry_liedtke]",[curt_a._stark],[richard_voß],"[henny_porten, frida_richard, hans_marr, harry_liedtke, drama, curt_a._stark, richard_voß]"
...,...,...,...,...,...,...,...
187645,the_rehearsal_tt9916170,the rehearsal,[drama],"[kelner_macêdo, pablo_lafuente, julia_ianina, germano_melo]",[tamar_guimaraes],"[melissa_de_raaf, lillah_halla]","[kelner_macêdo, pablo_lafuente, julia_ianina, germano_melo, drama, tamar_guimaraes, melissa_de_raaf, lillah_halla]"
187646,il_talento_del_calabrone_tt9916270,il talento del calabrone,[thriller],"[anna_foglietta, lorenzo_richelmy, sergio_castellitto, marina_occhionero]",[giacomo_cimini],[lorenzo_collalti],"[anna_foglietta, lorenzo_richelmy, sergio_castellitto, marina_occhionero, thriller, giacomo_cimini, lorenzo_collalti]"
187647,coven_tt9916362,coven,"[drama, history, horror]","[alex_brendemühl, daniel_fanego, amaia_aberasturi, garazi_urkola]",[pablo_agüero],[katell_guillou],"[alex_brendemühl, daniel_fanego, amaia_aberasturi, garazi_urkola, drama, history, horror, pablo_agüero, katell_guillou]"
187648,kuambil_lagi_hatiku_tt9916538,kuambil lagi hatiku,[drama],"[cut_mini_theo, sahil_shah, lala_karmela, ria_irawan]",[azhar_kinoi_lubis],"[rino_sarjono, arief_ash_siddiq, salman_aristo]","[cut_mini_theo, sahil_shah, lala_karmela, ria_irawan, drama, azhar_kinoi_lubis, rino_sarjono, arief_ash_siddiq, salman_aristo]"


In [22]:
#movies_df.to_csv('../models/exp4/imdb_doc2vec_train_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../models/exp4/imdb_doc2vec_train_data.csv'

# ********************************* Create the doc2vec model **********************

In [23]:
# Create the tagged document needed for Doc2Vec
tagged_documents=[]
movies_df.progress_apply(lambda row : tagged_documents.append(TaggedDocument(row.movie_sentence,[row.movie_id])),axis=1)

#convert the series of tagged docuemnt to a list
train_data = list(tagged_documents)

print("Length of training data {}".format(len(train_data)))


Length of training data 187650


In [24]:
train_data

[TaggedDocument(words=['jim_gerald', 'george_merriman', 'lance_vane', 'arthur_guest', 'rhoda_dendron', 'william_duff', 'drama', 'charles_macmahon', 'rolf_boldrewood'], tags=['robbery_under_arms_tt0000615']),
 TaggedDocument(words=['fernanda_negri_pouget', 'drama', 'mario_caserini', 'william_shakespeare'], tags=['hamlet_tt0000630']),
 TaggedDocument(words=['joaquín_carrasco', 'jaime_borrás', 'drama', 'narciso_cuyàs', 'duque_de_rivas'], tags=['don_álvaro_o_la_fuerza_del_sino_tt0000676']),
 TaggedDocument(words=['jean_mounet-sully', 'drama', 'gérard_bourgeois', 'william_shakespeare'], tags=['hamlet,_prince_of_denmark_tt0000886']),
 TaggedDocument(words=['joaquín_carrasco', 'josé_argelagués', 'elvira_fremont', 'josé_durany', 'drama', 'alberto_marro', 'ricardo_de_baños', 'manuel_tamayo_y_baus'], tags=['locura_de_amor_tt0000941']),
 TaggedDocument(words=['dante_cappelli', 'maria_caserini', 'drama', 'mario_caserini', 'william_shakespeare'], tags=['amleto_tt0001112']),
 TaggedDocument(words=['

In [25]:
random.seed = 400
[random.shuffle(data.words) for data in train_data]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [26]:
random.shuffle(train_data)

In [30]:
cores = multiprocessing.cpu_count()
model = gensim.models.doc2vec.Doc2Vec(
                dm= 0, dbow_words=1, vector_size= 100, 
                hs=0 ,min_count=5,
                workers=cores, epochs= 50, sample=0,
                ns_exponent=(-3), negative=20, window_size= 10000)

model.build_vocab(train_data)
model.train(train_data, total_examples=len(train_data) , epochs=model.epochs)


2020-12-07 18:58:33,966 : INFO : collecting all words and their counts
2020-12-07 18:58:33,968 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-12-07 18:58:34,257 : INFO : PROGRESS: at example #10000, processed 87337 words (303198/s), 53108 word types, 10000 tags
2020-12-07 18:58:34,363 : INFO : PROGRESS: at example #20000, processed 174538 words (827736/s), 91439 word types, 20000 tags
2020-12-07 18:58:34,443 : INFO : PROGRESS: at example #30000, processed 262167 words (1096585/s), 123724 word types, 30000 tags
2020-12-07 18:58:34,512 : INFO : PROGRESS: at example #40000, processed 349621 words (1283251/s), 152307 word types, 40000 tags
2020-12-07 18:58:34,599 : INFO : PROGRESS: at example #50000, processed 437271 words (1016948/s), 178046 word types, 50000 tags
2020-12-07 18:58:34,658 : INFO : PROGRESS: at example #60000, processed 524720 words (1508513/s), 202181 word types, 60000 tags
2020-12-07 18:58:34,735 : INFO : PROGRESS: at example #70000, 

In [31]:
#model = Doc2Vec.load('models/exp4/model') 
model.save("model")

2020-12-07 21:10:36,115 : INFO : saving Doc2Vec object under model, separately None
2020-12-07 21:10:36,117 : INFO : storing np array 'vectors_docs' to model.docvecs.vectors_docs.npy
2020-12-07 21:10:37,180 : INFO : saved model


In [ ]:
words_vectors = model.wv
#get the docs vectors represenation
docs_vectors = model.docvecs

In [ ]:
print(words_vectors.similarity("adam_sandler", "jennifer_aniston"))
print(words_vectors.similarity('douglas_hickox','cy_endfield'))

In [ ]:
sent = helper_fn.find_movie_sent_by_id('harry_potter_and_the_sorcerer\'s_stone_tt0241527',movies_df)
print(sent[0])
infered_movie = model.infer_vector(sent[0])
similar_to_infered = docs_vectors.most_similar([infered_movie], topn=20)

print(*similar_to_infered, sep='\n')

print(helper_fn.find_movie_by_id('alone_tt3838034', movies_df).movie_sentence)

In [ ]:
docs_vectors.most_similar(['spider-man_tt0145487'])

In [ ]:

tsne_model = TSNE(n_jobs=4,
                  early_exaggeration=4,
                  n_components=2,
                  verbose=1,
                  random_state=2018,
                  n_iter=300)
#tsne_d2v = tsne_model.fit_transform(model.docvecs.vectors_docs)



In [ ]:
doc_tags = list(model.docvecs.doctags.keys())
sample_tags= doc_tags[0:1000]
sample_docs = model[sample_tags]

In [ ]:
sample_tags

In [ ]:
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(sample_docs)
df = pd.DataFrame(X_tsne, index= sample_tags, columns=['x', 'y'])

In [ ]:
X = model.wv[model.wv.vocab][0:10000]

tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)

plt.scatter(X_tsne[:, 0], X_tsne[:, 1])
plt.show()

In [ ]:
X_tsne.

In [ ]:
tsne_d2v

plt.scatter(X_tsne[:, 0], X_tsne[:, 1])
plt.show()

In [ ]:
def tsne_plot_word2vec(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []
    

    for word in model.wv.vocab[0:5000]:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
movies_df.isnull().any()

In [ ]:
movies_df.describe

In [ ]:
def find_in_movie_sent(search_list, row):
 #   print(row)
    if all(elem in row['movie_sentence']  for elem in search_list):
        return(row['movie_id'])
    else:
        return False
        
search_list=[ 'adventure', 'family','chris_columbus', 'j.k._rowling']
x=movies_df.apply(lambda row: find_in_movie_sent(search_list, row),axis=1)
#doc_tags =[]
for item in x:
    if item != False:
        doc_tags.append(item)
        

    

In [ ]:
doc_tags

In [ ]:
def tsne_plot(sample_tags, model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []
    
    for doc in sample_tags:
        tokens.append(model[doc])
        labels.append(doc)
    
    tsne_model = TSNE(perplexity=5, n_components=2, init='pca', n_iter=10000, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
tsne_plot(list(set(tags)),model)
#docs_vectors.doctags

In [ ]:
#model['going_overboard_tt0096870']
sim = docs_vectors.most_similar([model['billy_madison_tt0112508']])

In [ ]:
tags=[] 
for id, similarity in sim:
    tags.append(id)

In [ ]:
print(*doc_tags, sep='\n')

In [ ]:
movie_sent_dict = dict(zip(movies_df.movie_id, movies_df.movie_sentence))

In [ ]:
target_id = 'the_hunger_games_tt1392170'
n_items= 5
treshold=.4
data_size= 10

topn_list = model.docvecs.most_similar([target_movie_id], topn= n_items + 1)
#remove the target movie from the list
for idx, sim in enumerate(topn_list):    
    if sim[0] == target_id:        
        topn_list.pop(idx)
#(topn_list.pop(idx) if (topn_list[0] == target_movie_id ) for idx, sim in enumerate(topn_list))
precision_df = calculate_precision(target_id, topn_list, movie_sent_dict, treshold)

In [ ]:
model.docvecs.most_similar([model[target_id]], topn=5+1)

In [ ]:
model.docvecs.most_similar([target_id], topn=5)

In [ ]:
random.seed = 400
n_items= 5
treshold=.44
data_size= 100
random.seed = 150
dict_keys = list(movie_sent_dict.keys())
random.shuffle(dict_keys)
movie_id_list = random.sample(dict_keys, data_size)
#movie_id_list=["the_hunger_games_tt1392170", "harry_potter_and_the_sorcerer's_stone_tt0241527"]
precision_df = calculate_precision_data_sample(5 , 4 ,movie_id_list, movie_sent_dict, treshold )
#movie_id_list

precision_df

In [ ]:
docs_vectors.most_similar('harry_potter_and_the_sorcerer\'s_stone_tt0241527')
docs_vectors.most_similar('survivor_tt3247714')

In [ ]:


movies_df.sample(frac=1)

In [ ]:
sample_movies_df = movies_df.loc[movies_df.apply(lambda movie: True if ('romance' in movie['genre']) 
                                                 and ('comedy' in movie.genre) and ('adam_sandler' in movie.actor_name) else False, axis=1)]

In [ ]:
sample_movies_df[['title','genre','actor_name','director_name','writer_name']]

In [ ]:
#drama_movies_df = movies_df.loc[movies_df.genre.apply(lambda genre: True if 'drama' in genre else False)]

common_attrs_df= pd.DataFrame(columns=['movie_id','actors_cnt', 'directors_cnt','writers_cnt','genres_cnt','total_cnt'])

sample_movies_df.reset_index(drop=True, inplace=True)
print(query_movie_id)
query_movie_id= sample_movies_df.iloc[2].movie_id

common_attrs_df= common_attrs_df.append(sample_movies_df.progress_apply(lambda row: find_common_attrs(sample_movies_df,query_movie_id, row.movie_id), axis=1)
                                        ,ignore_index = True)

print(common_attrs_df)

In [ ]:
sample_movies_df = movies_df.loc[movies_df.apply(lambda movie: True if ('romance' in movie['genre']) 
                                                 and ('comedy' in movie.genre) and ('adam_sandler' in movie.actor_name) else False, axis=1)]

common_attr_matrix = sample_movies_df.apply(lambda row: find_common_attrs_in_list(sample_movies_df, row.movie_id, cols), axis=1)
common_attr_matrix.set_index('title', inplace= True)

In [ ]:
common_attr_matrix

In [ ]:
#common_attr_matrix = common_attr_matrix.rename_axis(index=labels)


In [ ]:
common_attrs_df = common_attrs_df[common_attrs_df.movie_id != query_movie_id]

In [ ]:
common_attrs_df[common_attrs_df.total_cnt == common_attrs_df.total_cnt.max()]


In [ ]:
docs_vectors.most_similar('the_wedding_singer_tt0120888', topn=30)

In [ ]:
helper_fn.find_movie_sent_by_id('a_night_at_the_roxbury_tt0120770',movies_df)

In [ ]:
helper_fn.find_movie_sent_by_id('the_wedding_singer_tt0120888',movies_df)

In [ ]:
helper_fn.find_movie_sent_by_id('mr._deeds_tt0280590',movies_df)

In [ ]:
find_common_attrs(movies_df,'the_wedding_singer_tt0120888', 'mr._deeds_tt0280590')

In [ ]:
find_common_attrs(movies_df,'the_wedding_singer_tt0120888', 'a_night_at_the_roxbury_tt0120770')

In [ ]:
def find_common_attrs_dict(movies_df,query_movie_id, search_movie_id):
    query_movie= movies_df[movies_df.movie_id == query_movie_id]
    search_movie= movies_df[movies_df.movie_id == search_movie_id]
    
    common_actors= count_sim(query_movie, search_movie, 'actor_name')
    common_directors= count_sim(query_movie, search_movie, 'director_name')
    common_writers= count_sim(query_movie, search_movie, 'writer_name')
    common_genres= count_sim(query_movie, search_movie, 'genre')
    
    common_dict = {search_movie_id: common_actors+common_directors+common_writers+common_genres}
    return common_dict

In [ ]:
    common_attrs_dict=[]
    common_attrs_dict = [movies_df.progress_apply(lambda row: find_common_attr(movies_df,query_movie_id, row.movie_id), axis=1)]

In [ ]:
drama_movies_df = movies_df.loc[movies_df.genre.apply(lambda genre: True if 'drama' in genre else False)]

In [ ]:
len(drama_movies_df)

In [30]:
def start_param_search(train_data_df):
    #prepare train
    tagged_documents=[]
    train_data_df.progress_apply(lambda row : tagged_documents.append(TaggedDocument(row.movie_sentence,[row.movie_id])),axis=1)
    train_data = list(tagged_documents)
    random.seed = 400
    [random.shuffle(data.words) for data in train_data]
    random.shuffle(train_data)
    
    #prepare test data
    test_data= pd.read_csv("sequel_test_data.csv")
    #prepare parameters list
    top=3
    dm = [0]
    vector_size = [50,300]
    window = [12,100000]
    hs = [0]
    min_count=[1,5]
    epochs=[50]
    ns_exponent=[3,-3]
    dbow_words=[1]
    negative=[20]
    param_list = [{'dm': item[0],'vector_size': item[1],'window': item[2],'hs': item[3],'epochs':item[4], 'min_count':item[5],'ns_exponent':item[6], 'dbow_words':item[7],'negative':item[8] }
                  for item in
                         list(itertools.product(*[dm, vector_size, window, hs, epochs, min_count, ns_exponent, dbow_words, negative]))
                ]
    print(len(param_list))
    #print(len(param_list))
    models_scores = eval_fn.model_parameters_search (train_data, test_data, param_list, train_data_df, top)

    score_doc2vec = pd.DataFrame(models_scores)
 #   score_doc2vec = score_doc2vec.sort_values(by=['rank_score'], ascending=False)
    return score_doc2vec
    #call the search function
    

In [31]:
eval_score_df = start_param_search(movies_df)

2020-12-06 10:06:18,523 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-12-06 10:06:18,524 : INFO : collecting all words and their counts
2020-12-06 10:06:18,524 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-12-06 10:06:18,601 : INFO : PROGRESS: at example #10000, processed 87266 words (1149319/s), 52716 word types, 10000 tags
2020-12-06 10:06:18,659 : INFO : PROGRESS: at example #20000, processed 174827 words (1520525/s), 90974 word types, 20000 tags
2020-12-06 10:06:18,718 : INFO : PROGRESS: at example #30000, processed 262468 words (1514014/s), 122844 word types, 30000 tags


16
training model with the parameters {'dm': 0, 'vector_size': 50, 'window': 12, 'hs': 0, 'epochs': 50, 'min_count': 1, 'ns_exponent': 3, 'dbow_words': 1, 'negative': 20}


2020-12-06 10:06:18,772 : INFO : PROGRESS: at example #40000, processed 349922 words (1640766/s), 151407 word types, 40000 tags
2020-12-06 10:06:18,843 : INFO : PROGRESS: at example #50000, processed 437584 words (1260130/s), 177443 word types, 50000 tags
2020-12-06 10:06:18,903 : INFO : PROGRESS: at example #60000, processed 525259 words (1474979/s), 201741 word types, 60000 tags
2020-12-06 10:06:18,960 : INFO : PROGRESS: at example #70000, processed 612911 words (1509310/s), 224403 word types, 70000 tags
2020-12-06 10:06:19,028 : INFO : PROGRESS: at example #80000, processed 700552 words (1309423/s), 246071 word types, 80000 tags
2020-12-06 10:06:19,090 : INFO : PROGRESS: at example #90000, processed 787875 words (1421344/s), 266597 word types, 90000 tags
2020-12-06 10:06:19,154 : INFO : PROGRESS: at example #100000, processed 875471 words (1399663/s), 286318 word types, 100000 tags
2020-12-06 10:06:19,223 : INFO : PROGRESS: at example #110000, processed 962751 words (1273662/s), 304

An exception occured while trying to evaluate the model with the params {'dm': 0, 'vector_size': 50, 'window': 12, 'hs': 0, 'epochs': 50, 'min_count': 1, 'ns_exponent': 3, 'dbow_words': 1, 'negative': 20} raised exception: '<' not supported between instances of 'str' and 'int'
training model with the parameters {'dm': 0, 'vector_size': 50, 'window': 12, 'hs': 0, 'epochs': 50, 'min_count': 1, 'ns_exponent': -3, 'dbow_words': 1, 'negative': 20}


2020-12-06 10:19:59,342 : INFO : PROGRESS: at example #30000, processed 262468 words (1319834/s), 122844 word types, 30000 tags
2020-12-06 10:19:59,417 : INFO : PROGRESS: at example #40000, processed 349922 words (1160752/s), 151407 word types, 40000 tags
2020-12-06 10:19:59,494 : INFO : PROGRESS: at example #50000, processed 437584 words (1164338/s), 177443 word types, 50000 tags
2020-12-06 10:19:59,561 : INFO : PROGRESS: at example #60000, processed 525259 words (1314479/s), 201741 word types, 60000 tags
2020-12-06 10:19:59,625 : INFO : PROGRESS: at example #70000, processed 612911 words (1393105/s), 224403 word types, 70000 tags
2020-12-06 10:19:59,690 : INFO : PROGRESS: at example #80000, processed 700552 words (1361318/s), 246071 word types, 80000 tags
2020-12-06 10:19:59,765 : INFO : PROGRESS: at example #90000, processed 787875 words (1158380/s), 266597 word types, 90000 tags
2020-12-06 10:19:59,840 : INFO : PROGRESS: at example #100000, processed 875471 words (1183253/s), 28631

...

2020-12-06 14:56:47,921 : INFO : PROGRESS: at example #10000, processed 87266 words (96439/s), 52716 word types, 10000 tags
2020-12-06 14:56:47,990 : INFO : PROGRESS: at example #20000, processed 174827 words (1282264/s), 90974 word types, 20000 tags
2020-12-06 14:56:48,043 : INFO : PROGRESS: at example #30000, processed 262468 words (1681007/s), 122844 word types, 30000 tags
2020-12-06 14:56:48,103 : INFO : PROGRESS: at example #40000, processed 349922 words (1489988/s), 151407 word types, 40000 tags
2020-12-06 14:56:48,167 : INFO : PROGRESS: at example #50000, processed 437584 words (1360060/s), 177443 word types, 50000 tags
2020-12-06 14:56:48,223 : INFO : PROGRESS: at example #60000, processed 525259 words (1575180/s), 201741 word types, 60000 tags
2020-12-06 14:56:48,278 : INFO : PROGRESS: at example #70000, processed 612911 words (1632587/s), 224403 word types, 70000 tags
2020-12-06 14:56:48,333 : INFO : PROGRESS: at example #80000, processed 700552 words (1595738/s), 246071 word

An exception occured while trying to evaluate the model with the params {'dm': 0, 'vector_size': 300, 'window': 100000, 'hs': 0, 'epochs': 50, 'min_count': 5, 'ns_exponent': 3, 'dbow_words': 1, 'negative': 20} raised exception: '<' not supported between instances of 'str' and 'int'
training model with the parameters {'dm': 0, 'vector_size': 300, 'window': 100000, 'hs': 0, 'epochs': 50, 'min_count': 5, 'ns_exponent': -3, 'dbow_words': 1, 'negative': 20}


2020-12-06 15:09:53,621 : INFO : PROGRESS: at example #40000, processed 349922 words (1518309/s), 151407 word types, 40000 tags
2020-12-06 15:09:53,698 : INFO : PROGRESS: at example #50000, processed 437584 words (1145246/s), 177443 word types, 50000 tags
2020-12-06 15:09:53,753 : INFO : PROGRESS: at example #60000, processed 525259 words (1595468/s), 201741 word types, 60000 tags
2020-12-06 15:09:53,809 : INFO : PROGRESS: at example #70000, processed 612911 words (1586030/s), 224403 word types, 70000 tags
2020-12-06 15:09:53,881 : INFO : PROGRESS: at example #80000, processed 700552 words (1229664/s), 246071 word types, 80000 tags
2020-12-06 15:09:53,943 : INFO : PROGRESS: at example #90000, processed 787875 words (1418824/s), 266597 word types, 90000 tags
2020-12-06 15:09:54,000 : INFO : PROGRESS: at example #100000, processed 875471 words (1555837/s), 286318 word types, 100000 tags
2020-12-06 15:09:54,055 : INFO : PROGRESS: at example #110000, processed 962751 words (1609604/s), 304

An exception occured while trying to evaluate the model with the params {'dm': 0, 'vector_size': 300, 'window': 100000, 'hs': 0, 'epochs': 50, 'min_count': 5, 'ns_exponent': -3, 'dbow_words': 1, 'negative': 20} raised exception: '<' not supported between instances of 'str' and 'int'


KeyError: 'rank_score'

In [ ]:
sparql = SPARQLWrapper2("http://lod.openlinksw.com/sparql/")  
sparql.setQuery(query) 
sparql.setReturnFormat(JSON)   
sparql.setMethod(POST)
results = sparql.query()

In [32]:
eval_score

NameError: name 'eval_score' is not defined